# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.experiment import Experiment
import os

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-ml'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-203997
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-203997


In [3]:
key = "forestfire"
dataset = ws.datasets[key] 

df = dataset.to_pandas_dataframe()
df.describe()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes_mapped
count,243.000000,243.000000,243.0,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000
mean,15.761317,7.502058,2012.0,32.152263,62.041152,15.493827,0.762963,77.842387,14.680658,49.430864,4.742387,16.690535,7.035391,0.563786
std,8.842552,1.114793,0.0,3.628039,14.828160,2.811385,2.003207,14.349641,12.393040,47.665606,4.154234,14.228421,7.440568,0.496938
min,1.000000,6.000000,2012.0,22.000000,21.000000,6.000000,0.000000,28.600000,0.700000,6.900000,0.000000,1.100000,0.000000,0.000000
25%,8.000000,7.000000,2012.0,30.000000,52.500000,14.000000,0.000000,71.850000,5.800000,12.350000,1.400000,6.000000,0.700000,0.000000
50%,16.000000,8.000000,2012.0,32.000000,63.000000,15.000000,0.000000,83.300000,11.300000,33.100000,3.500000,12.400000,4.200000,1.000000
75%,23.000000,8.000000,2012.0,35.000000,73.500000,17.000000,0.500000,88.300000,20.800000,69.100000,7.250000,22.650000,11.450000,1.000000
max,31.000000,9.000000,2012.0,42.000000,90.000000,29.000000,16.800000,96.000000,65.900000,220.400000,19.000000,68.000000,31.100000,1.000000


In [5]:
df.head(5)

,Column2,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes_mapped
0,0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,0
1,1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,0
2,2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,0
3,3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,0
4,4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,0


## Create or Attach an AmlCompute cluster

In [6]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "hyperdrive-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1, delay_evaluation=5 )

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "C": choice(0.1, 1, 5, 10),
    "max_iter": choice(500, 1000, 4000, 10000)
}
)

#TODO: Create your estimator and hyperdrive config
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".", script="train.py", compute_target=compute_target, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, hyperparameter_sampling=param_sampling, policy=early_termination_policy, primary_metric_name="Accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, max_concurrent_runs=4)

In [19]:
#TODO: Submit your experiment

from azureml.core.experiment import Experiment
experiment = Experiment(ws, "hyperdrive1")
run = experiment.submit(config=hyperdrive_config)
run.wait_for_completion(show_output=True)

RunId: HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e
Web View: https://ml.azure.com/runs/HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-203997/workspaces/quick-starts-ws-203997&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-08-11T16:07:43.043161][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-08-11T16:07:43.8658732Z][SCHEDULER][INFO]Scheduling job, id='HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e_0' 
[2022-08-11T16:07:44.0160552Z][SCHEDULER][INFO]Scheduling job, id='HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e_1' 
[2022-08-11T16:07:44.1177215Z][SCHEDULER][INFO]Scheduling job, id='HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e_2' 
[2022-08-11T16:07:44.232685][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-08-11T16:07:44.2887358Z][SCHEDULER][INFO]Scheduling job, id='HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de0

{'runId': 'HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e',
 'target': 'hyperdrive-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-08-11T16:07:42.260611Z',
 'endTimeUtc': '2022-08-11T16:09:45.192635Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '11dfae34-b9ab-4bb6-bb3d-8ec56ac9fd59',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1083-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.42.0',
  'space_size': '16',
  'score': '0.9452054794520548',
  'best_child_run_id': 'HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'aml

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [21]:
import joblib
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
best_run.get_details()

{'runId': 'HD_2d8dbdfb-1654-4d3c-a234-d5e31c0de01e_0',
 'target': 'hyperdrive-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-08-11T16:07:57.864563Z',
 'endTimeUtc': '2022-08-11T16:08:13.811829Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '11dfae34-b9ab-4bb6-bb3d-8ec56ac9fd59',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '1', '--max_iter', '1000'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'hyperdrive-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identi

In [22]:
best_run_metrics = best_run.get_metrics()
print('Accuracy:', best_run_metrics["Accuracy"])

Accuracy: 0.9452054794520548


In [ ]:
src_bestrun = ScriptRunConfig(source_directory=".", script="train.py", compute_target=cpu_cluster, environment=sklearn_env,arguments=['--C',0.1, '--max_iter',1000])
experiment = Experiment(ws, "bestrun1")
run2 = experiment.submit(config=src_bestrun)
run2.wait_for_completion(show_output=True)

In [24]:
#TODO: Save the best model
best_run.download_file("model.pkl")

In [23]:
model = best_run.register_model(model_name="model.pkl")
print(model.name, model.id, model.version, sep='\t')

model.pkl	model.pkl:1	1


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
print(run2.get_file_names())

# register model 
model = run2.register_model(model_name='hyperdrive_model', model_path='outputs/hyperdrivemodel.pkl')
print(model.name, model.id, model.version, sep='\t')

# create environment for the deploy
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice

# get a curated environment
env = Environment.get(
    workspace=ws, 
    name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference",
    version=1
)
env.inferencing_stack_version='latest'

# create deployment config i.e. compute resources
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description="hyperdrive_for_email",
)


%%time
import uuid
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.model import Model

# get the registered model
model = Model(ws, "outputs/hyperdrivemodel.pkl")

# create an inference config i.e. the scoring script and environment
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# deploy the service
service_name = "hyperdrive_email"
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
)

service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# send raw HTTP request to test the web service.
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X) - 1)
input_data = '{"data": [' + str(list(X[random_index])) + "]}"

headers = {"Content-Type": "application/json"}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("label:", y[random_index])
print("prediction:", resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
resp.raise_for_status()

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

